In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

from sklearn.metrics import mean_absolute_error
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn import preprocessing
import xgboost as xgb

#### Modelo: XGB

In [2]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat] = temp #Dejo en el DF
    
    return train

In [3]:
temp = train.groupby("tipodepropiedad")["precio"].agg(np.mean) #Saco promedio
#temp = preprocessing.scale(temp) #Normalizo
#temp
#temp_test = test.groupby("tipodepropiedad")["tipodepropiedad"]
#.transform(decode, temp = temp, label = "tipodepropiedad")
temp_test = test['tipodepropiedad'].map(temp)
temp_test = preprocessing.scale(temp_test)
temp_test

array([-0.41126792,  0.71062843,  0.71062843, ..., -0.41126792,
       -0.41126792, -0.41126792])

In [4]:
def mean_target_decoding(test, nombrefeat, nombretarget, train):
    
    temp = train.groupby(nombrefeat)[nombretarget].agg(np.mean) #Saco promedio
    temp_test = test[nombrefeat].map(temp)
    temp_test = preprocessing.scale(temp_test)
    test[nombrefeat] = temp_test

    return test

In [5]:
train = mean_target_encoding(train, "tipodepropiedad", "precio")
train = mean_target_encoding(train, "provincia", "precio")
train = mean_target_encoding(train, "ciudad", "precio")

#Adiciono servicios
train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']

#Adiciono cercanias
train['ubicacion'] = train['escuelascercanas'] + train['centroscomercialescercanos']

#Elimino los residuos
train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                      'centroscomercialescercanos'], axis = 1)

#Armo mi nuevo train
trainXGB = train.drop(['descripcion', 'titulo', 'direccion', 'fecha'], axis = 1)

In [6]:
x_train = trainXGB.drop(['precio'], axis = 1) 
y_train = trainXGB['precio']

#Armo modelo
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",
                             colsample_bytree = 0.6900510179471959,
                             gamma = 0.11042626041444276,
                             learning_rate = 0.036505998002060044,
                             max_depth = 12,
                             n_estimators = 400,
                             subsample = 0.6194353865754738,
                             eval_metric = "mae") #Que es la que se usa en la competencia

In [7]:
import timeit
start_time = timeit.default_timer()


#Fitteo
xgb_model.fit(x_train, y_train)


elapsed = timeit.default_timer() - start_time

/home/ailen-magali/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ailen-magali/.local/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [8]:
elapsed

617.8633453630027

In [ ]:
#Preparo TEST
trainoriginal = pd.read_csv("train.csv")
test = mean_target_decoding(test, "tipodepropiedad", "precio", trainoriginal)
test = mean_target_decoding(test, "provincia", "precio", trainoriginal)
test = mean_target_decoding(test, "ciudad", "precio", trainoriginal)

#Adiciono servicios
test['servicios'] = test['piscina'] + test['gimnasio'] + test['usosmultiples']

#Adiciono cercanias
test['ubicacion'] = test['escuelascercanas'] + test['centroscomercialescercanos']

#Elimino los residuos
test = test.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 'fecha',
                  'centroscomercialescercanos', 'titulo', 'direccion', 'descripcion'], axis = 1)

#Predigo
prediccion = xgb_model.predict(test)

In [ ]:
predicciones = test.loc[:, ['id','ciudad']]
predicciones.rename(columns = {'ciudad' : 'target'}, inplace = True)
predicciones['target'] = prediccion
predicciones = predicciones.set_index('id')

In [ ]:
predicciones.to_csv(r'prediccionXGB4.csv')